In [1]:
import torch
from torch.optim import SGD, Adam
from AlexNet import AlexNet
from helper import load_tiny_imagenet
from helper import evaluate_accuracy, evaluate_loss


%load_ext autoreload
%autoreload 2

In [2]:
# Download: http://cs231n.stanford.edu/tiny-imagenet-200.zip
train, val, test = load_tiny_imagenet().values()

C:\tools\Anaconda3\envs\pytorch\lib\site-packages\torchvision\transforms\transforms.py:210: UserWarning: The use of the transforms.Scale transform is deprecated, please use transforms.Resize instead.
  warnings.warn("The use of the transforms.Scale transform is deprecated, " +


In [3]:
torch.cuda.empty_cache()

In [4]:
alexNet_sgd = AlexNet(100, verbose=True)
alexNet_adam = AlexNet(100, verbose=True)

In [9]:
"""
We trained our models using stochastic gradient descent with a batch size of 128 examples,
    momentum of 0.9, and weight decay of 0.0005.
We used an equal learning rate for all layers, which we adjusted manually throughout training.
The heuristic which we followed was to divide the learning rate by 10 when the validation
    error rate stopped improving with the current learning rate.
The learning rate was initialized at 0.01 and reduced three times prior to termination.
"""
LR = 0.01
p = 0.9
decay = 0.0005

# Original paper uses SGD as the optimizer, which doesn't converge=...
sgd_optimizer = SGD(alexNet_sgd.parameters(), lr=LR, momentum=p, weight_decay=decay)
adam_optimizer = Adam(alexNet_adam.parameters(), lr=0.0001)

scheduler_sgd = torch.optim.lr_scheduler.ReduceLROnPlateau(
        optimizer=sgd_optimizer,
        mode='min', factor=0.1, patience=5, threshold=0.002
)

scheduler_adam = torch.optim.lr_scheduler.ReduceLROnPlateau(
        optimizer=adam_optimizer,
        mode='min', factor=0.1, patience=5, threshold=1e-4
)

In [ ]:
alexNet_sgd.initialize(optimizer=sgd_optimizer, scheduler=scheduler_sgd)
alexNet_sgd.train(mode=True, data=train, epochs=20)

In [10]:
alexNet_adam.initialize(optimizer=adam_optimizer, scheduler=scheduler_adam)
alexNet_adam.train(mode=True, data=train, epochs=20)

[epoch 1, batch 200] loss: 7.135173358917236
[epoch 1, batch 400] loss: 4.607536435127258
[epoch 1, batch 600] loss: 4.60124048948288
[epoch 1, batch 800] loss: 4.597749187946319


KeyboardInterrupt: 